<a href="https://colab.research.google.com/github/ivelin/canswim/blob/main/canswim_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an experimental attempt to implement growth stock indicators in support of CANSLIM style investors and traders.

Check whether GPU is available

In [ ]:
# import torch
# torch.cuda.is_available()

Load data from prepared local csv file

In [ ]:
from darts import TimeSeries
import pandas as pd


In [ ]:
series = TimeSeries.from_csv('data/market_data.csv', time_col='Date')

Prepare train and validation splits

In [ ]:
val_start = pd.Timestamp('2023-05-05')
train, val = series.split_before(val_start)


In [ ]:
len(train), len(val)

In [ ]:
train.head()

In [ ]:
train_df = train.pd_dataframe()


In [ ]:
train_df.shape[0] - train_df.dropna().shape[0]

In [ ]:
val.head()

In [ ]:
val_df = val.pd_dataframe()
val_df

In [ ]:
val_df.shape[0] - val_df.dropna().shape[0]

Plot data splits

In [ ]:
import matplotlib.pyplot as plt
 
train.plot(label='train')
val.plot(label='val', lw=3)
plt.legend()

Show seasonalities

In [ ]:
from darts.utils.statistics import plot_acf, check_seasonality

plot_acf(train, alpha=0.05)

Train a baseline model to benchmark against

In [ ]:
from darts.models import ExponentialSmoothing


baseline_model = ExponentialSmoothing()
baseline_model.fit(train)
prediction = baseline_model.predict(len(val), num_samples=500)

In [ ]:
import matplotlib.pyplot as plt
 

actual = series.slice(val_start-pd.Timedelta(weeks=52), series.end_time())
actual.plot(label='actual')
prediction.plot(label='forecast', lw=3)
plt.legend()

In [ ]:
#from darts.models import AutoARIMA
#  
#model_aarima = AutoARIMA()
#model_aarima.fit(train)
#prediction_aarima = model_aarima.predict(len(val))

In [ ]:
#import matplotlib.pyplot as plt
# 
#actual.plot(label='actual')
#prediction_aarima.plot(label='forecast', lw=3)
# plt.legend()

Train our current working model

In [ ]:
#from darts.dataprocessing.transformers import Scaler
# 
# scaler = Scaler()
#train_air_scaled = scaler.fit_transform(train)
# 
#train_air_scaled.plot()
# 
# encoders = {"datetime_attribute": {"past": ["month", "year"]}} # , "transformer": Scaler()}

In [ ]:
train_history = 252 # 252 days in a year with market data
pred_horizon = 21*2 # 21 days in a month with market data
n_epochs = 100 # model training epochs

In [ ]:
from darts.models import TiDEModel

from darts.utils.likelihood_models import QuantileRegression

saved_model_name = 'data/canswim_model.pt'

# If available, load the saved model state that performed best on validation set
try:
  print('Loading saved model')
  model = TiDEModel.load(saved_model_name)  
except Exception as e:
  print('Error loading saved model:', e)
  print('Proceeding with new model')
  model = TiDEModel(
      input_chunk_length=train_history,
      output_chunk_length=pred_horizon,
      # add_encoders=encoders,
      dropout=0.1,
      n_epochs=n_epochs,
      use_reversible_instance_norm=True,
      likelihood=QuantileRegression(quantiles=[0.01, 0.05, 0.2, 0.5, 0.8, 0.95, 0.99])
#      model_name=saved_model_name,
#      force_reset=True,
#      save_checkpoints=True,    
  )  



In [ ]:
from darts.models import RegressionModel
from darts.models.forecasting.forecasting_model import GlobalForecastingModel

# when True, multiple time series are supported
supports_multi_ts = issubclass(model.__class__, GlobalForecastingModel)

In [ ]:
supports_multi_ts

In [ ]:
# train model

model.fit(train, epochs=n_epochs)


In [ ]:
# save model
model.save(saved_model_name)


In [ ]:
# use probabilistic prediction
# see https://unit8co.github.io/darts/userguide/forecasting_overview.html#probabilistic-forecasts
pred = model.predict(pred_horizon, mc_dropout=True, num_samples=500) # len(val))

In [ ]:
actual.plot(label='actual')
pred.plot(label='forecast', lw=3)
plt.legend()

Backtest Model


In [ ]:

#models = [ExponentialSmoothing(),
##          NHiTSModel(input_chunk_length=6,
##            output_chunk_length=6),
#          TiDEModel(
#            input_chunk_length=24,
#            output_chunk_length=12,
#            use_reversible_instance_norm=True),
#            ]
#
#backtests = [model.historical_forecasts(series,
#                            start=.5,
#                            forecast_horizon=3)
#             for model in models]

In [ ]:
# from darts.metrics import mape
#
#series.plot(label='data')
#for i, m in enumerate(models):
#    err = mape(backtests[i], series)
#    backtests[i].plot(lw=3, label='{}, MAPE={:.2f}%'.format(m, err))
#
#plt.title('Backtests with 3-months forecast horizon')
#plt.legend()

Backtest model on the full range of test data

In [ ]:
from darts.metrics import rmse

def eval_model(model=None, series=None, start=None, past_covariates=None, future_covariates=None, forecast_horizon=None):
    # Past and future covariates are optional because they won't always be used in our tests
    
    # We backtest the model on the last 20% of the flow series, with a horizon of 10 steps:
    backtest = model.historical_forecasts(series=series, 
                                          past_covariates=past_covariates,
                                          future_covariates=future_covariates,
                                          start=start, 
                                          retrain=False,
                                          verbose=True, 
                                          forecast_horizon=forecast_horizon,
                                          num_samples=500 # probabilistic forecasting
                                          )
    
    return backtest


In [ ]:
backtest = eval_model(model=model, series=series, start=val_start, forecast_horizon=pred_horizon)
actual.plot(label='actual')
backtest.plot(label='backtest (n=10)')
print('Backtest RMSE = {}'.format(rmse(series, backtest)))